In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('e:/data/churn.csv')

In [ ]:
# Assuming we already have done detailed analysis on this data

In [ ]:
# Class distribution of the taget variable

In [ ]:
df['Exited'].value_counts()

In [ ]:
x = df.iloc[:,2:12]

In [ ]:
y = df['Exited']

In [ ]:
dummies = pd.get_dummies(data=df, columns=['Geography','Gender'], drop_first=True)

In [ ]:
dummies[:2]

In [ ]:
dummies.columns

In [ ]:
x=pd.concat([x,dummies[['Geography_Germany', 'Geography_Spain', 'Gender_Male']]], axis=1)

In [ ]:
x.drop(['Geography','Gender'], axis=1, inplace=True)

In [ ]:
x.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=2)

In [ ]:
ytrain.value_counts()

In [ ]:
6346/8000

In [ ]:
1654/2000

In [ ]:
# Generalised Linear models

In [ ]:
# Logistic regression --> Logit function (sigmoid function) --> odds

In [ ]:
p/(1-p)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
glm = LogisticRegression(max_iter=10000,solver="liblinear")

In [ ]:
lrmodel = glm.fit(xtrain,ytrain)

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
# ---- Predict probabilities ----
y_prob = lrmodel.predict_proba(xtest)[:, 1]
fpr, tpr, thresholds = roc_curve(ytest, y_prob)
roc_auc = auc(fpr, tpr)
print("AUC Score:", roc_auc)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--', label='Random Guess')
plt.xlabel("False Positive Rate") 
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

In [ ]:
lrpred= lrmodel.predict(xtest)

In [ ]:
pd.crosstab(ytest,lrpred)

In [ ]:
(1591+13)/2000

In [ ]:
13/(13+26)

In [ ]:
13/(13+370)

In [ ]:
lrmodel.predict(xtest[:1])

In [ ]:
lrmodel.predict_proba(xtest[:1])

In [ ]:
ytest[:1]

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [ ]:
dtree = DecisionTreeClassifier(max_depth=5, min_samples_leaf=5, min_samples_split=5, max_leaf_nodes=8, random_state=8)

In [ ]:
dtreemodel = dtree.fit(xtrain,ytrain)

In [ ]:
# Graphviz
# use dtreeviz, plotly, dtreeplt for advanced visualisation

In [ ]:
from dtreeplt import dtreeplt

In [ ]:
x.columns

In [ ]:
plt.figure(figsize=(20, 10))
plot_tree(
    dtreemodel,
    filled=True,
    feature_names=x.columns,
    class_names=['pos','neg'],
    rounded=True,
    fontsize=10
)
plt.show()

In [ ]:
dtree_pred = dtreemodel.predict(xtest)

In [ ]:
pd.crosstab(ytest,dtree_pred)

In [ ]:
92/(92+24)

In [ ]:
# Precision --> tp/(tp+fp)
146/(146+63)

In [ ]:
# Recall --> tp/(tp+fn)
146/(237+146)

In [ ]:
# Accuracy
(146+1554)/2000

In [ ]:
# Pruning

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score

In [ ]:
from sklearn import tree

In [ ]:
params = {'max_depth': [4,6,8,10],
         'min_samples_split': [2,5,8],
         'min_samples_leaf': [4,10]}

clf = tree.DecisionTreeClassifier()
gcv = GridSearchCV(estimator=dtreemodel,param_grid=params)
gcv.fit(xtrain,ytrain)

In [ ]:
gcv.cv_results_

In [ ]:
gcv.best_params_

In [ ]:
gcv.best_score_

In [ ]:
path = dtree.cost_complexity_pruning_path(xtrain,ytrain)

In [ ]:
alpha = path['ccp_alphas']

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, classification_report

In [ ]:
ytrain.pred

In [ ]:
alpha

In [ ]:
acc_train, acc_test = [],[]

for i in alpha:
    tree = DecisionTreeClassifier(ccp_alpha=i)
    tree.fit(xtrain,ytrain)
    ytrain_pred = tree.predict(xtrain)
    ytest_pred = tree.predict(xtest)
    
    acc_train.append(accuracy_score(ytrain,ytrain_pred))
    acc_test.append(accuracy_score(ytest,ytest_pred))
    acc_train.append(precision_score(ytrain,ytrain_pred))
    acc_test.append(precision_score(ytest,ytest_pred))

In [ ]:
(92)/(92+291)

In [ ]:
acc_train

In [ ]:
acc_test

In [ ]:
m1 = DecisionTreeClassifier(ccp_alpha=0.00522287)
m1.fit(xtrain,ytrain)

In [ ]:
p=m1.predict(xtest)

In [ ]:
pd.crosstab(ytest,p)

In [ ]:
print(classification_report(ytest,p))

In [ ]:
precision_score(ytest,p)